In [1]:
import cv2
from ultralytics import YOLO
from roboflow import Roboflow

model = YOLO('yolov8n.pt')

/Users/kenjohnson/Documents/Projects/Personal/Table Tennis/tt_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
rf = Roboflow(api_key="7ncFBCdmUBerMMm67kXx")
project = rf.workspace("cpsc-440").project("ball-detection-ofm6m")
dataset = project.version(2).download("yolov5")

loading Roboflow workspace...
loading Roboflow project...


Extracting Dataset Version Zip to Ball-Detection-2 in yolov5pytorch:: 100%|██████████| 538/538 [00:00<00:00, 1140.36it/s]


Make sure you change the train and val directories in the data.yaml (remove )

In [22]:
dataset.location

'/Users/kenjohnson/Documents/Projects/Personal/Table Tennis/Ball-Detection-1'

In [5]:
model.train(data=f"{dataset.location}/data.yaml", epochs=10, imgsz=640)

New https://pypi.org/project/ultralytics/8.0.88 available 😃 Update with 'pip install -U ultralytics'
Ultralytics YOLOv8.0.82 🚀 Python-3.9.15 torch-1.13.0 CPU
yolo/engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/Users/kenjohnson/Documents/Projects/Personal/Table Tennis/Ball-Detection-2/data.yaml, epochs=10, patience=50, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=None, exist_ok=False, pretrained=False, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, image_weights=False, rect=False, cos_lr=False, close_mosaic=0, resume=False, amp=True, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, show=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, vid_stride=1, line_thickness=3, visualize=False, augment=False, agnostic_nms

In [7]:
results = model.val(data=f"{dataset.location}/data.yaml")

Ultralytics YOLOv8.0.82 🚀 Python-3.9.15 torch-1.13.0 CPU
val: Scanning /Users/kenjohnson/Documents/Projects/Personal/Table Tennis/Ball-Detection-2/valid/labels.cache... 52 images, 9 backgrounds, 0 corrupt: 100%|██████████| 52/52 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:27<00:00,  6.88s/it]
                   all         52         43          1     0.0233      0.512      0.205
Speed: 19.1ms preprocess, 450.3ms inference, 0.0ms loss, 0.8ms postprocess per image
Results saved to /Users/kenjohnson/Documents/Projects/Personal/Table Tennis/runs/detect/val


In [6]:
results = model.predict(source=f'{dataset.location}/test/images', save=True)


image 1/27 /Users/kenjohnson/Documents/Projects/Personal/Table Tennis/Ball-Detection-2/test/images/0-0-111_jpeg.rf.c940040f62a23cb961f2ebac738db421.jpg: 640x640 (no detections), 1690.9ms
image 2/27 /Users/kenjohnson/Documents/Projects/Personal/Table Tennis/Ball-Detection-2/test/images/0-0-116_jpeg.rf.a65ce00cbd95595d4b9f452c8af7c908.jpg: 640x640 (no detections), 1110.0ms
image 3/27 /Users/kenjohnson/Documents/Projects/Personal/Table Tennis/Ball-Detection-2/test/images/0-0-117_jpeg.rf.e617257d18259db9db3abd2544b163fa.jpg: 640x640 (no detections), 825.5ms
image 4/27 /Users/kenjohnson/Documents/Projects/Personal/Table Tennis/Ball-Detection-2/test/images/0-0-123_jpeg.rf.32a7cc0f935f4efad5476a6f810257ed.jpg: 640x640 (no detections), 1025.5ms
image 5/27 /Users/kenjohnson/Documents/Projects/Personal/Table Tennis/Ball-Detection-2/test/images/0-0-126_jpeg.rf.84e6ff98bb5560230d8447dfcfe9f619.jpg: 640x640 (no detections), 940.3ms
image 6/27 /Users/kenjohnson/Documents/Projects/Personal/Table Ten

In [22]:
rf = Roboflow(api_key="7ncFBCdmUBerMMm67kXx")
project = rf.workspace().project("ball-detection-ofm6m")
model = project.version(2).model

loading Roboflow workspace...
loading Roboflow project...


In [ ]:
model.predict('')

In [21]:
from PIL import Image, ImageDraw, ImageFont
import os
for filename in os.listdir('Ball-Detection-2/test/images'):
    results = model.predict(f'Ball-Detection-2/test/images/{filename}', labels=True, stroke=100).predictions
    f = f'Ball-Detection-2/test/images/{filename}'
    image = Image.open(f)

    draw = ImageDraw.Draw(image)
    font = ImageFont.load_default()

    for box in results:
        color = "#FF0000"
        x1 = box['x'] - box['width'] / 2
        x2 = box['x'] + box['width'] / 2
        y1 = box['y'] - box['height'] / 2
        y2 = box['y'] + box['height'] / 2

        draw.rectangle([
            x1, y1, x2, y2
        ], outline=color, width=1)

    image.save(f'Ball-Detection-2/test/predictions/{filename}')

In [2]:
res_plotted = results[0].plot()
cv2.imwrite("result.jpg", res_plotted)

True

In [7]:
for i, result in enumerate(results):
    res_plotted = result.plot()
    cv2.imwrite(f"./yolov8test/{i}.png", res_plotted)

KeyboardInterrupt: 